### Домашнее задание 6

Сгенерировать последовательности, которые бы состояли из цифр (от 0 до 9)
и задавались следующим образом:

x - последовательность цифр

y1 = x1, y(i) = x(i) + x(1). 

Если y(i) >= 10, то y(i) = y(i) - 10

Задача:
научить модель предсказывать y(i) по x(i)
пробовать RNN, LSTM, GRU

In [1]:
import random
import torch
import time

In [2]:
vocab = [int(char) for char in '0123456789']

In [3]:
num_examples = 128
message_length = 5


def dataset(num_examples):
    dataset = []
    
    for ex in range(num_examples):     
        x = [random.choice(vocab) for x in range(message_length)]
        
        y = [x[0]]
        for xi in x[1:]:
            if (xi + x[0]) >= 10:
                y.append((xi + x[0])%10)
            else:
                y.append(xi + x[0])        
        
        dataset.append([torch.tensor(x), torch.tensor(y)])
    return dataset

In [5]:
dataset(num_examples)[0]

[tensor([1, 5, 5, 9, 2]), tensor([1, 6, 6, 0, 3])]

#### Задача: научить модель предсказывать y(i) по x(i)

In [6]:
embedding_dim = 10
hidden_dim = 10
vocab_size = len(vocab)

num_epochs = 100

##### RNN

In [7]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        ## Здесь создать слои
        self.embed = torch.nn.Embedding(vocab_size, embedding_dim)
        self.rnn = torch.nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.linear = torch.nn.Linear(hidden_dim, vocab_size)
        
    def forward(self, sentences, state=None):
        embed = self.embed(sentences)
        o, s = self.rnn(embed)
        out = self.linear(o)
        return out

In [8]:
model = Network()

In [9]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=.05)

In [10]:
for ep in range(num_epochs):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for x, y in dataset(num_examples):
        
        optimizer.zero_grad()
        answers = model.forward(x.unsqueeze(1))
        answers = answers.view(-1, vocab_size)
        loss = criterion(answers, y)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

Epoch 0. Time: 0.435, Train loss: 2.360
Epoch 1. Time: 0.354, Train loss: 2.408
Epoch 2. Time: 0.352, Train loss: 2.441
Epoch 3. Time: 0.343, Train loss: 2.422
Epoch 4. Time: 0.384, Train loss: 2.344
Epoch 5. Time: 0.438, Train loss: 2.473
Epoch 6. Time: 0.337, Train loss: 2.393
Epoch 7. Time: 0.319, Train loss: 2.448
Epoch 8. Time: 0.350, Train loss: 2.432
Epoch 9. Time: 0.799, Train loss: 2.381
Epoch 10. Time: 0.577, Train loss: 2.435
Epoch 11. Time: 0.461, Train loss: 2.412
Epoch 12. Time: 0.582, Train loss: 2.396
Epoch 13. Time: 0.306, Train loss: 2.437
Epoch 14. Time: 0.310, Train loss: 2.475
Epoch 15. Time: 0.287, Train loss: 2.446
Epoch 16. Time: 0.289, Train loss: 2.409
Epoch 17. Time: 0.291, Train loss: 2.456
Epoch 18. Time: 0.329, Train loss: 2.433
Epoch 19. Time: 0.412, Train loss: 2.416
Epoch 20. Time: 0.350, Train loss: 2.393
Epoch 21. Time: 0.353, Train loss: 2.425
Epoch 22. Time: 0.348, Train loss: 2.462
Epoch 23. Time: 0.350, Train loss: 2.419
Epoch 24. Time: 0.349, Tra

In [11]:
with torch.no_grad():
        matches, total = 0, 0
        for x, y in dataset(num_examples):
            answers = model.forward(x.unsqueeze(1))
            predictions = torch.nn.functional.softmax(answers, dim=2)
            _, batch_out = predictions.max(dim=2)
            batch_out = batch_out.squeeze(1)
            matches += torch.eq(batch_out, y).sum().item()
            total += torch.numel(batch_out)
        accuracy = matches / total
        print('Accuracy: {:4.2f}%'.format(accuracy * 100))

Accuracy: 15.78%


##### GRU

In [12]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        ## Здесь создать слои
        self.embed = torch.nn.Embedding(vocab_size, embedding_dim)
        self.rnn = torch.nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.linear = torch.nn.Linear(hidden_dim, vocab_size)
        
    def forward(self, sentences, state=None):
        embed = self.embed(sentences)
        o, s = self.rnn(embed)
        out = self.linear(o)
        return out

In [13]:
model = Network()

In [14]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=.05)

In [15]:
for ep in range(num_epochs):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for x, y in dataset(num_examples):
        
        optimizer.zero_grad()
        answers = model.forward(x.unsqueeze(1))
        answers = answers.view(-1, vocab_size)
        loss = criterion(answers, y)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

Epoch 0. Time: 0.398, Train loss: 2.297
Epoch 1. Time: 0.408, Train loss: 2.280
Epoch 2. Time: 0.454, Train loss: 2.261
Epoch 3. Time: 0.499, Train loss: 2.289
Epoch 4. Time: 0.451, Train loss: 2.277
Epoch 5. Time: 0.389, Train loss: 2.205
Epoch 6. Time: 0.682, Train loss: 2.286
Epoch 7. Time: 0.793, Train loss: 2.244
Epoch 8. Time: 0.413, Train loss: 2.313
Epoch 9. Time: 0.341, Train loss: 2.296
Epoch 10. Time: 0.350, Train loss: 2.211
Epoch 11. Time: 0.370, Train loss: 2.256
Epoch 12. Time: 0.652, Train loss: 2.261
Epoch 13. Time: 0.523, Train loss: 2.285
Epoch 14. Time: 0.361, Train loss: 2.247
Epoch 15. Time: 0.400, Train loss: 2.211
Epoch 16. Time: 0.413, Train loss: 2.305
Epoch 17. Time: 0.491, Train loss: 2.299
Epoch 18. Time: 0.417, Train loss: 2.331
Epoch 19. Time: 0.406, Train loss: 2.257
Epoch 20. Time: 0.510, Train loss: 2.319
Epoch 21. Time: 0.394, Train loss: 2.283
Epoch 22. Time: 0.409, Train loss: 2.247
Epoch 23. Time: 0.380, Train loss: 2.291
Epoch 24. Time: 0.421, Tra

In [16]:
with torch.no_grad():
        matches, total = 0, 0
        for x, y in dataset(num_examples):
            answers = model.forward(x.unsqueeze(1))
            predictions = torch.nn.functional.softmax(answers, dim=2)
            _, batch_out = predictions.max(dim=2)
            batch_out = batch_out.squeeze(1)
            matches += torch.eq(batch_out, y).sum().item()
            total += torch.numel(batch_out)
        accuracy = matches / total
        print('Accuracy: {:4.2f}%'.format(accuracy * 100))

Accuracy: 14.22%


##### LSTM

In [17]:
embed = torch.nn.Embedding(vocab_size, embedding_dim)
lstm = torch.nn.LSTM(embedding_dim, hidden_dim)
linear = torch.nn.Linear(hidden_dim, vocab_size)
softmax = torch.nn.functional.softmax
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(list(embed.parameters()) +
                             list(lstm.parameters()) +
                             list(linear.parameters()), lr=0.001)

In [18]:
def zero_hidden():
    return (torch.zeros(1, 1, hidden_dim),
            torch.zeros(1, 1, hidden_dim))

In [19]:
accuracies, max_accuracy = [], 0
for ep in range(num_epochs):
    print('Epoch: {}'.format(ep))
    for x, y in dataset(num_examples):
        lstm_in = embed(x)
        lstm_in = lstm_in.unsqueeze(1)
        lstm_out, lstm_hidden = lstm(lstm_in, zero_hidden())
        scores = linear(lstm_out)
        scores = scores.transpose(1, 2)
        y = y.unsqueeze(1)
        loss = loss_fn(scores, y) 
        loss.backward()
        optimizer.step()
    print('Loss: {:6.4f}'.format(loss.item()))
Epoch: 0

Epoch: 0
Loss: 2.3471
Epoch: 1
Loss: 2.1080
Epoch: 2
Loss: 2.0135
Epoch: 3
Loss: 2.2321
Epoch: 4
Loss: 2.2405
Epoch: 5
Loss: 1.3513
Epoch: 6
Loss: 2.3529
Epoch: 7
Loss: 2.3158
Epoch: 8
Loss: 2.0253
Epoch: 9
Loss: 1.8456
Epoch: 10
Loss: 1.7184
Epoch: 11
Loss: 2.4171
Epoch: 12
Loss: 1.8004
Epoch: 13
Loss: 2.1888
Epoch: 14
Loss: 1.3082
Epoch: 15
Loss: 1.7454
Epoch: 16
Loss: 1.5385
Epoch: 17
Loss: 1.3441
Epoch: 18
Loss: 1.5405
Epoch: 19
Loss: 2.5192
Epoch: 20
Loss: 2.1221
Epoch: 21
Loss: 2.4481
Epoch: 22
Loss: 2.3196
Epoch: 23
Loss: 2.0616
Epoch: 24
Loss: 1.9251
Epoch: 25
Loss: 1.6729
Epoch: 26
Loss: 1.1197
Epoch: 27
Loss: 1.8560
Epoch: 28
Loss: 1.8745
Epoch: 29
Loss: 0.9057
Epoch: 30
Loss: 2.5699
Epoch: 31
Loss: 1.7453
Epoch: 32
Loss: 1.3882
Epoch: 33
Loss: 2.2684
Epoch: 34
Loss: 1.8724
Epoch: 35
Loss: 1.1253
Epoch: 36
Loss: 1.5858
Epoch: 37
Loss: 1.9263
Epoch: 38
Loss: 1.2725
Epoch: 39
Loss: 2.6276
Epoch: 40
Loss: 3.0020
Epoch: 41
Loss: 1.8808
Epoch: 42
Loss: 1.9027
Epoch: 43
Loss: 2.267

In [20]:
with torch.no_grad():
        matches, total = 0, 0
        for x, y in dataset(num_examples):
            lstm_in = embed(x)
            lstm_in = lstm_in.unsqueeze(1)
            lstm_out, lstm_hidden = lstm(lstm_in, zero_hidden())
            scores = linear(lstm_out)
            predictions = softmax(scores, dim=2)
            _, batch_out = predictions.max(dim=2)
            batch_out = batch_out.squeeze(1)
            matches += torch.eq(batch_out, y).sum().item()
            total += torch.numel(batch_out)
        accuracy = matches / total
        print('Accuracy: {:4.2f}%'.format(accuracy * 100))

Accuracy: 29.69%


len 100 100e RNN 15.78% GRU 14.22% LSTM 29.69%